In [1]:
import logging
import json
from datetime import datetime, timedelta
import pandas as pd
#from config import configuration
import concurrent.futures
from olx_downloader import OlxDownloader
import urllib3

In [41]:
def download_rent_listings(filters, max_pages=25):
    base_url = 'https://www.olx.uz/api/v1/offers/'
    limit = 50
    offset = 0
    url = base_url + '?offset=' + str(offset) + '&' + filters
    count = 0
    listings = []
    page = 0
    ids = set()
    # last_refresh_time = None
    while url:
        #last_refresh_time = None
        res = download_page(url)
        url = None
        page = page + 1
        if 'data' in res:
            data = res['data']
            i = 0
            for listing in data:
                i = i + 1
                if 'id' in listing:
                    if listing['id'] not in ids:
                        ids.add(listing['id'])
                        listings.append(listing)
                        #promoted_top = listing['promotion']['top_ad']
                        # if not promoted_top:
                        #     if (last_refresh_time is None) or (last_refresh_time > listing['last_refresh_time']):
                        #         last_refresh_time = listing['last_refresh_time']
            count = count + i
        if 'links' in res:
            if 'next' in res['links']:
                if 'href' in res['links']['next']:
                    url = res['links']['next']['href']
        if (page == max_pages):
            print(f'Max pages reached, count: {len(listings)}, filter: {filters}')
            #url = None
        # if (last_refresh_time < min_last_refresh_time):
        #     print(f'Min date reached, count: {len(listings)}')
        #     url = None
        #print(f'page: {page}, min last_refresh_time on the page: {last_refresh_time}')
    logging.info('Downloaded %s items', count)
    return listings

def download_page(url):
    http = urllib3.PoolManager()
    r = http.request('GET', url)
    #print(r.data)
    return json.loads(r.data.decode('utf-8'))

In [42]:
def build_filter_url(filter_base, room_min, room_max, floor_min, floor_max, price_min, price_max):
    rooms_filter = f'filter_float_number_of_rooms%3Afrom={room_min}&filter_float_number_of_rooms%3Ato={room_max}'
    floor_filter = f'filter_float_floor%3Afrom={floor_min}&filter_float_floor%3Ato={floor_max}'
    price_filter = f'filter_float_price%3Afrom={price_min}&filter_float_price%3Ato={price_max}'
    filter_url = f'{filter_base}&{rooms_filter}&{floor_filter}&{price_filter}'
    return filter_url

In [43]:
room_filters = [
    (1,1),
    (2,2),
    (3,3),
    (4,4),
    (5,5),
    (6,100)
]

floor_filters = [
    (1,1),
    (2,2),
    (3,3),
    (4,4),
    (5,5),
    (6,6),
    (7,7),
    (8,8),
    (9,9),
    (10,10),
    (11,11),
    (12,12),
    (13,13),
    (14,14),
    (15,20),
    (20,100)
]

price_filters = [
    (200,300),
    (300.01,400),
    (400.01,500),
    (500.01,600),
    (600.01,700),
    (700.01,800),
    (800.01,900),
    (900.01,1000),
    (1000.01,1100),
    (1100.01,1200),
    (1200.01,1300),
    (1300.01,1400),
    (1400.01,100000)
]

In [44]:
# price_filters = []
# price_min = 300
# price_max = 2000
# #price_inf = 100000
# price_step = 25
# price_eps = 1
# for p in range(price_min,price_max,price_step):
#     price_filters.append((p, p + price_step - price_eps))
# #price_filters.append((price_max, price_inf))

In [45]:
filter_base = 'category_id=1147&region_id=5&city_id=4&currency=USD&filter_refiners=&sl=184915fd305x3316b009'
filters_list = []
for (rooms_min, rooms_max) in room_filters:
    for floor_min, floor_max in floor_filters:
        for price_min, price_max in price_filters:
            filter_url = build_filter_url(filter_base, rooms_min, rooms_max, floor_min, floor_max, price_min, price_max)
            filters_list.append(filter_url)

In [46]:
len(filters_list)

1248

In [47]:
# downloaded_listings = []
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     future_to_filter = {executor.submit(download_rent_listings, filters, 120): filters for filters in filters_list}
#     for future in concurrent.futures.as_completed(future_to_filter):
#         filter = future_to_filter[future]
#         try:
#             downloaded_listings = downloaded_listings + future.result()
#         except Exception as exc:
#             print('%r generated an exception: %s' % (filter, exc))
    #futures = [
    #    executor.submit(download_rent_listings, filters)
    #    for filters in filters_list]
    #for future in futures:
    #    if len(future.result()) > 0:
    #        #print('result: ' + str(future.result()[0]))
    #        try:
    #            downloaded_listings = downloaded_listings + future.result()
    #        except Exception as exc:
    #            print(exc)
#with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
#        for listings_chunk in executor.map(download_rent_listings, filters_list):
#            print(listings_chunk)

In [ ]:
#f = 'category_id=1147&region_id=5&city_id=4&currency=USD&filter_refiners=&sl=184915fd305x3316b009&filter_float_number_of_rooms%3Afrom=5&filter_float_number_of_rooms%3Ato=5&filter_float_floor%3Afrom=4&filter_float_floor%3Ato=4&filter_float_price%3Afrom=700.01&filter_float_price%3Ato=800'
#download_rent_listings(f)

In [48]:
len_filters = len(filters_list)
downloaded_listings = []
for i, filters in enumerate(filters_list):
    if (i+1) % 10 == 0:    
        print(f'Downloading batch {i+1} of {len_filters}')
    listings = download_rent_listings(filters)
    downloaded_listings = downloaded_listings + listings

In [49]:
len(downloaded_listings)

35967

In [50]:
data = json.dumps(downloaded_listings).encode('UTF-8')
filename = f'oxl_download_rent.json'
with open(filename,'w') as fp:
   json.dump(downloaded_listings, fp)

'category_id=1147&region_id=5&city_id=4&currency=USD&filter_refiners=&sl=184915fd305x3316b009&sort_by=created_at%3Adesc&filter_float_number_of_rooms%3Afrom=1&filter_float_number_of_rooms%3Ato=2&filter_float_floor%3Afrom=1&filter_float_floor%3Ato=100&filter_float_price%3Afrom=200&filter_float_price%3Ato=500'